# bge-reranker-v2-m3

> 作者: 黎颖; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn;   l.tingting@pku.edu.cn


本节旨在展示使用 reranker 模型来计算语言相似度的流程，使用的模型为 bge-reranker-v2-m3。

分以下几步来实现：
1. 环境安装与应用创建
2. 下载模型
3. 模型使用

    3.1 通过 FlagEmbedding 运行

    3.2 通过 Huggingface Transformer 运行


reranker 使用 cross similarity 来计算两个句子的相似度，把两个句子共同作为模型的输入，与 embedding 方法相比更耗费计算资源，但是对语言有更好的理解。

## 1. 环境安装与应用创建

首先在data shell中创建conda环境:
```bash
conda create -n tutorial5 python=3.9
conda activate tutorial5
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install numpy==1.26.4 matplotlib==3.8.4 ipykernel==6.29.5 transformers==4.42.4
```

然后创建JupyterLab应用, `Conda环境名`请填写`tutorial5`, 硬件资源为1个GPU。创建应用后, 进入应用并打开本文件。

## 2. 下载模型

在联网的机器上下载模型：

```bash
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download BAAI/bge-reranker-v2-m3 --local-dir models--BAAI--bge-reranker-v2-m3
```

如果使用 SCOW， 可以从公用存储中加载模型，存储所有模型的路径为：

/lustre/public/tutorial/models/models--BAAI--bge-reranker-v2-m3

从中找到本节对应的模型加载即可。

## 3. 模型使用

[[参考链接]](https://huggingface.co/BAAI/bge-reranker-v2-m3)

### 3.1 通过 FlagEmbedding 运行

使用 FlagEmbedding， 计算 cross similarity

安装 FlagEmbedding 包：

```bash
pip install -U FlagEmbedding
```

安装后，运行下方代码：

In [ ]:
from FlagEmbedding import FlagReranker

# 填写模型路径
# VAR_PLACEHOLDER
reranker = FlagReranker('models--BAAI--bge-reranker-v2-m3', use_fp16=True) 

# 计算相似度
score = reranker.compute_score(['query', 'passage'], normalize=True)
print(score) # 0.003497010252573502

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], normalize=True)
print(scores) # [0.00027803096387751553, 0.9948403768236574]

### 3.2 通过 Huggingface Transformer 运行

或者使用 Huggingface Transformer 也可以调用模型，运行下方代码：

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 填写模型路径
# VAR_PLACEHOLDER
modle_path = 'models--BAAI--bge-reranker-v2-m3'

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(modle_path)
model = AutoModelForSequenceClassification.from_pretrained(modle_path)
model.eval()

# 把模型移动到显卡
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


# 计算相似度
pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    
    # 将输入数据移动到GPU
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)